<a href="https://colab.research.google.com/github/ta3map/StudentNeuralNetwork/blob/main/StudentsNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predict next grades from previous

Example data

In [117]:
!git clone https://github.com/ta3map/StudentNeuralNetwork

fatal: destination path 'StudentNeuralNetwork' already exists and is not an empty directory.


In [2]:
import pandas as pd
import numpy as np

df = pd.read_excel('/content/StudentNeuralNetwork/example_dataset.xltx', skiprows = 0)

# make all columns as str
df.columns = df.columns.astype(str)

df

,year,2018,2018.1,2018.2,2019,2019.1,2019.2,2019.3,2020,2020.1,2020.2
0,student,math,physics,history,computer science,literature,music,electricity,management,economics,circuit design
1,1,5,4,3,5,3,3,5,3,3,4
2,2,4,4,4,4,3,4,4,4,4,4
3,3,2,3,5,2,5,5,2,2,5,3
4,4,5,5,4,5,4,4,5,5,4,5
5,5,3,4,3,3,3,3,4,3,3,3
6,6,5,4,3,4,5,3,4,3,3,5
7,7,3,3,4,3,4,4,3,4,4,3
8,8,4,2,2,2,3,2,4,2,2,4
9,9,4,4,4,4,4,4,4,4,4,4


Choose the year

In [3]:
year = 2018
# Find column whose name contains a specific string - year
spike_cols = [col for col in df.columns if str(year) in col]
print(list(df.columns))
print(spike_cols)

['year', '2018', '2018.1', '2018.2', '2019', '2019.1', '2019.2', '2019.3', '2020', '2020.1', '2020.2']
['2018', '2018.1', '2018.2']


In [4]:
# extract grades from this year
df[spike_cols].to_numpy()[1:-1].astype(int)

array([[5, 4, 3],
       [4, 4, 4],
       [2, 3, 5],
       [5, 5, 4],
       [3, 4, 3],
       [5, 4, 3],
       [3, 3, 4],
       [4, 2, 2],
       [4, 4, 4]])

In [5]:
df[spike_cols].to_numpy()[0].astype(str)

array(['math', 'physics', 'history'], dtype='<U7')

a function for getting data from the year

In [29]:
# function for getting data from the year
def getYear(df, year):
  spike_cols = [col for col in df.columns if str(year) in col]
  data_out = df[spike_cols].to_numpy()[1:-1].astype(int)
  columns = df[spike_cols].to_numpy()[0].astype(str)
  return data_out, columns

first_year, first_columns = getYear(df, 2018)
second_year, second_columns = getYear(df, 2019)
third_year, third_columns = getYear(df, 2020)

ouput_size = np.size(second_year, 1)
input_size = np.size(first_year, 1)
R = [];
j = 0 # output index 
for i in range(input_size):  # input index
  r = np.corrcoef(first_year[:, i], second_year[:, j])
  R.append(r[0, 1])

print('how ', first_columns, ' predicts', second_columns[j], ':')
print(R)

how  ['math' 'physics' 'history']  predicts computer science :
[0.7926290870042669, 0.8512565307587486, 0.02786391062876759]


In [150]:
from numpy.linalg import norm

# 'music' from 'math' and 'history'
R = np.array([-0.5,  1, 0.5, 0.6, -1])
grades_in = np.array([5, 4, 5, 2, 4])

Rp = R.copy()
#Rp[Rp<0] = 0
p_cond = np.where(Rp>0)
Rbp = Rp[p_cond]/norm(Rp[p_cond])
Rp[Rp<0] = 0

Rn = R.copy()
n_cond = np.where(Rn<0)
Rbn = Rn[n_cond]/norm(Rn[n_cond])

print(R[p_cond], norm(Rbp*grades_in[p_cond]), R[n_cond], norm(Rbn*grades_in[n_cond]))
grade_out = norm(Rp*grades_in)/norm(Rp)
grade_out

[1.  0.5 0.6] 3.8359204520278722 [-0.5 -1. ] 4.219004621945797


3.8359204520278722

Predictive function for all grades

In [153]:
first_year, first_columns = getYear(df, 2018)
second_year, second_columns = getYear(df, 2019)

def learnGrades(first_year, second_year):
  ouput_size = np.size(second_year, 1)
  input_size = np.size(first_year, 1)
  R2 = [];
  for j in range(ouput_size): # output index 
    R1 = [];  
    for i in range(input_size):  # input index
      r = np.corrcoef(first_year[:, i], second_year[:, j])
      R1.append(r[0, 1])
    R2.append(R1);
  return np.array(R2)

R2 = learnGrades(first_year, second_year)
print(R2)

def predictGrades(grades_in, R2):
  ouput_size = np.size(R2, 0)
  G = [];
  for j in range(ouput_size):
    R = R2[j]
    R[R<0] = 0# avoid negative correlation
    grade_out = norm(R*grades_in)/norm(R)
    G.append(grade_out)
  return np.array(G)

grades_in = np.array([2, 4, 4])

print('how ', first_columns, grades_in, ' predicts')
print(second_columns, ':')
predictGrades(grades_in, R2)

[[ 0.79262909  0.85125653  0.02786391]
 [-0.17392527  0.05773503  0.52915026]
 [-0.46315109  0.27277236  1.        ]
 [ 0.88034084  0.57032343 -0.52610428]]
how  ['math' 'physics' 'history'] [2 4 4]  predicts
['computer science' 'literature' 'music' 'electricity'] :


array([3.22964701, 4.        , 4.        , 2.74727504])

Second year

In [172]:

# data in
first_year, first_columns = getYear(df, 2018)
second_year, second_columns = getYear(df, 2019)
third_year, third_columns = getYear(df, 2020)

# first step learning
grades_in = np.array([3, 3, 5])# 'math' 'physics' 'history'
R2 = learnGrades(first_year, second_year)
predicted_second_year = predictGrades(grades_in, R2)

# second step learning
grades_in = np.concatenate((grades_in, predicted_second_year))
previous_grades = np.concatenate((first_year, second_year), 1)
previous_grades_columns = np.concatenate((first_columns, second_columns), 0)
R2 = learnGrades(previous_grades, third_year)
print(R2)
predicted_third_year = predictGrades(grades_in, R2)

print('how ', previous_grades_columns, grades_in, ' predicts')
print(third_columns, ':', predicted_third_year)
print(predicted_third_year.astype(int))

[[ 0.39528471  0.72168784  0.33071891  0.70034929 -0.05        0.33071891
   0.44901326]
 [-0.46315109  0.27277236  1.          0.02786391  0.52915026  1.
  -0.52610428]
 [ 0.89331482  0.49236596 -0.26189246  0.64441198  0.1492405  -0.26189246
   0.67010963]]
how  ['math' 'physics' 'history' 'computer science' 'literature' 'music'
 'electricity'] [3.         3.         5.         3.00152907 4.9811409  4.88794095
 3.        ]  predicts
['management' 'economics' 'circuit design'] : [3.33626057 4.89861332 3.03063222]
[3 4 3]
